### MAC에 spark 설치 요망

- RDD는 두 가지 연산으로 이뤄져 있다.
- Transformation 
- Action

- Transformation -> Lazy Execution 또는 Lazy Loading
- 트랜스포메이션이 행해지면, RDD가 수행되는 것이 아닌, 새로운 RDD를 만들어 내고 
- 그 새로운 RDD에 수행 결과를 저장하게 된다.

- Action
- 메서드로 이루어진 실행 작업이며, 트랜스포메이션이 행해지고 나서 이뤄지는 Evaluation 작업

In [2]:
from pyspark import SparkConf , SparkContext
from pyspark.sql import SQLContext

In [3]:
conf = SparkConf().setMaster('local').setAppName('sparkApp')
spark = SparkContext(conf=conf)
spark

<SparkContext master=local appName=sparkApp>

In [4]:
rdd = spark.textFile('/Users/HojinChoi/Desktop/data/test.txt')
rdd

/Users/HojinChoi/Desktop/data/test.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [5]:
lines = rdd.filter(lambda x : 'spark' in x)
lines 

PythonRDD[2] at RDD at PythonRDD.scala:53

In [6]:
sample_rdd = spark.parallelize(['test' , 'this is a test rdd'])

- RDD 자주 쓰는 연산 함수
- collect() : RDD에 트랜스포메이션된 결과를 리턴하는 함수
- map() : 연산 수행하고 싶을 때 사용하는 함수

In [9]:
numbers = spark.parallelize(list(range(5)))
numbers

ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195

In [10]:
s = numbers.map(lambda x : x * x).collect()
s

[0, 1, 4, 9, 16]

- flatmap() : 리스트들의 원소를 하나의 리스트로 flatten 해서 리턴하는 함수

In [16]:
strings = spark.parallelize(['hi spark' , 'hi python' , 'hi django' , 'hi sklearn'])
flat_string = strings.flatMap(lambda x : x.split(' ')).collect()
flat_string 

['hi', 'spark', 'hi', 'python', 'hi', 'django', 'hi', 'sklearn']

- filter() : 조건으로 필터링하는 연산자

In [18]:
num = spark.parallelize(list(range(1, 30, 3)))
num

ParallelCollectionRDD[15] at parallelize at PythonRDD.scala:195

In [19]:
result = num.filter(lambda x : x % 2 == 0).collect()
result

[4, 10, 16, 22, 28]

### Pair RDD
- pair rdd 란 key-value 쌍으로 이뤄진 RDD
- python tuple 을 의미

In [20]:
# pair rdd 생성
pairRDD = spark.parallelize([(1, 3) , (1, 5) , (2, 4) , (3 , 3)])
pairRDD

ParallelCollectionRDD[17] at parallelize at PythonRDD.scala:195

- reduceByKey()

In [22]:
{
    i : j
    for i , j in pairRDD.reduceByKey(lambda x, y : x+y).collect()
}

{1: 8, 2: 4, 3: 3}

In [23]:
{
    i : j
    for i , j in pairRDD.mapValues(lambda x, y : x**2).collect()
}

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\i\Anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 377, in main
  File "C:\Users\i\Anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 372, in process
  File "C:\Users\i\Anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\i\Anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "C:\Users\i\Anaconda3\lib\site-packages\pyspark\rdd.py", line

In [24]:
pairRDD.keys().collect()

[1, 1, 2, 3]

In [25]:
pairRDD.values().collect()

[3, 5, 4, 3]

In [26]:
pairRDD.sortByKey().collect()

[(1, 3), (1, 5), (2, 4), (3, 3)]

### 외부 데이터를 로드해서 RDD 생성하는 방법

In [28]:
customerRDD = spark.textFile(/Users/HojinChoi/Desktop/data/spark-rdd-name-customers.csv)
customerRDD

SyntaxError: invalid syntax (<ipython-input-28-f25c3fbfe8cb>, line 1)

In [29]:
customerRDD.first()

NameError: name 'customerRDD' is not defined

In [ ]:
# map 연산자를 이용해 콤마로 split 하고 튜플로 리턴하는 구문 작성
cusPairs = customerRDD.map(lambda x : (x.split(',')[0] , x.split(',')[1] ))
cusPairs.collect()

In [ ]:
# groupByKey() : 키값을 리스트 형태로 리턴하는 함수
cusPairs.groupByKey().collect()

In [ ]:
# UK에 사는 고객 이름만 출력한다면? (dict 형식으로 만들어)
groupKey = cusPairs.groupByKey().collect()
#groupKey

for country , name in groupKey :
    if country == 'UK' :
        for name in names :
            print(name)

customerDict = {
    country  : [name for name in names] for country  , names in groupKey
}
customerDict['UK']

In [ ]:
- sortByKey : Key 오름차순으로 정렬하고 상위 10개만 뽑는다면?
cusPairs.sortByKey().collect()[:10]

In [ ]:
# 나라별 고객이 몇 명인지 카운트한다면?
# cusPairs.collect()
mapR = RcusPairs.mapValues(lambda x : 1).reduceByKey(lambda x , y : x+y)
type( mapR.collect())
{
    i : j
    for i , j in mapR.collect()
}

### Spark DataFrame

In [ ]:
- spark dataframe 은 RDD 의 확장된 구조이다
- 행 , 열로 이뤄진 내장 RDD

In [ ]:
- 생성
- 스파크 세션을 이용한 생성
- SQL 컨텍스트의 테이블을 통한 생성

In [ ]:
from pyspark import SparkConf , SparkContext
from pyspark.sql import SQLContext
import json

In [ ]:
sqlCtx = SQLContext(spark)
sqlCtx

In [ ]:
# json 파일
# json -> RDD -> DataFrame
sample_json = spark.textFile('/Users/HojinChoi/Desktop/data/cars.json')
cars_df = sqlCtx.createDataFrame(sample_json.map(lambda x : json.loads(x)))
cars_df.collect() 

In [ ]:
# csv 파일

In [ ]:
cars_df.printSchema()

In [ ]:
cars_df.show()

In [2]:
# 데이터 프레임에 대한 연산
# select()
cars_df.select('brand').show()

NameError: name 'cars_df' is not defined

In [ ]:
cars_df.select('models.price').show()

In [ ]:
# 컬럼의 타입 변환
from pyspark.sql.types import IntegerType

In [ ]:
car_price_type = cars_df.select('brand' , 'models.name', 'models.price')
# car_price_type
car_price_type = cars_price_type.withColumn('price' , cars_price_type['price'].cast(IntegerType))

In [ ]:
# cars_price_type.printSchema()
cars_price_type.first()

In [ ]:
# 비교 연산
# cars_price_type.collect()
cars_price_type.filter(cars_price_type['price'] > 20000).show()

In [ ]:
# 그룹핑
cars_price_type.groupBy('brand').count().show()